# Pre-Processing

In [1]:
#import

import pandas as pd
import io
import os
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import datetime as dt
from datetime import datetime
from sklearn.metrics import accuracy_score


In [ ]:
# Outline to connect to database (Not Operational At moment)
os.environ["AWS_ACCESS_KEY_ID"]= "xxxxxxxx"
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxxxxxxx"

s3_client = boto3.client('s3')
response = s3_client.get_object(Bucket="my_bucket", Key="filename.txt")
file = response["Body"]

pf.read_csv(file, header=14, delimiter="\t", low_memory=False)

In [27]:
# Create pathway to mock data
path = Path("clean_crime_mockup2.csv")

In [28]:
# load dataframe
crime_df = pd.read_csv(path)
crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [29]:
crime_df = crime_df[crime_df["Year"] >= 2015]

In [30]:
# Taking Count
crime_df.count()

ID                      1948783
Case Number             1948783
Date                    1948783
Block                   1948783
IUCR                    1948783
Primary Type            1948783
Description             1948783
Location Description    1940659
Arrest                  1948783
Domestic                1948783
Beat                    1948783
District                1948782
Ward                    1948731
Community Area          1948782
FBI Code                1948783
X Coordinate            1918378
Y Coordinate            1918378
Year                    1948783
Updated On              1948783
Latitude                1918378
Longitude               1918378
Location                1918378
dtype: int64

In [31]:
# Drop na values
crime_df = crime_df.dropna()

In [32]:
# Validate drops in na
crime_df.count()

ID                      1912868
Case Number             1912868
Date                    1912868
Block                   1912868
IUCR                    1912868
Primary Type            1912868
Description             1912868
Location Description    1912868
Arrest                  1912868
Domestic                1912868
Beat                    1912868
District                1912868
Ward                    1912868
Community Area          1912868
FBI Code                1912868
X Coordinate            1912868
Y Coordinate            1912868
Year                    1912868
Updated On              1912868
Latitude                1912868
Longitude               1912868
Location                1912868
dtype: int64

In [33]:
# dropping unnessary columns ( atbitrary placeholders)
cleaned_crime_df = crime_df.drop(columns = ['Case Number', 'ID', 'Block', 'Description', 'Year', 'IUCR', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Updated On', 'Location', 'Community Area', 'District', 'Ward', 'Beat', 'Domestic'])

In [34]:
# Extracting the hour from the Date column and then dropping the date column
cleaned_crime_df["Hour"] = pd.to_datetime(cleaned_crime_df["Date"]).dt.hour
cleaned_crime_df = cleaned_crime_df.drop(columns = ["Date"])

In [35]:
cleaned_crime_df.head()

,Primary Type,Location Description,Arrest,Latitude,Longitude,Hour
0,BATTERY,RESIDENCE,False,41.815117,-87.670000,13
1,THEFT,CTA BUS,False,41.895080,-87.765400,11
3,NARCOTICS,SIDEWALK,True,41.937406,-87.716650,12
4,ASSAULT,APARTMENT,False,41.881903,-87.755121,13
5,BURGLARY,RESIDENCE,False,41.744379,-87.658431,10


In [36]:
#Binning the time data
bins = [-1, 3, 7, 11, 15, 19, 24]
cleaned_crime_df['Time_binned'] = pd.cut(cleaned_crime_df['Hour'],
                                         bins=bins, 
                                         labels = [1,2,3,4,5,6])

In [37]:
# Dropping the hour column in favor of Time_binned
cleaned_crime_df = cleaned_crime_df.drop(columns = ["Hour"])

In [38]:
cleaned_crime_df.head()

,Primary Type,Location Description,Arrest,Latitude,Longitude,Time_binned
0,BATTERY,RESIDENCE,False,41.815117,-87.670000,4
1,THEFT,CTA BUS,False,41.895080,-87.765400,3
3,NARCOTICS,SIDEWALK,True,41.937406,-87.716650,4
4,ASSAULT,APARTMENT,False,41.881903,-87.755121,4
5,BURGLARY,RESIDENCE,False,41.744379,-87.658431,3


In [45]:
#Adding labelencoder for Primaty Type of crime
le = LabelEncoder()
cleaned_crime_df['Primary Type'] = le.fit_transform(cleaned_crime_df['Primary Type'])
cleaned_crime_df['Location Description'] = le.fit_transform(cleaned_crime_df['Location Description'])

In [46]:
cleaned_crime_df.head(10)

,Primary Type,Location Description,Arrest,Latitude,Longitude,Time_binned
0,2,145,False,41.815117,-87.670000,4
1,33,63,False,41.895080,-87.765400,3
3,18,166,True,41.937406,-87.716650,4
4,1,17,False,41.881903,-87.755121,4
5,3,145,False,41.744379,-87.658431,3
6,3,150,False,41.914636,-87.681631,5
7,33,96,True,41.851989,-87.689219,4
8,30,171,False,41.882814,-87.704326,3
10,33,135,False,41.763648,-87.722345,4
11,2,167,False,41.975968,-87.768014,3


# RandomForest Model

In [47]:
# Define the features set.
X = cleaned_crime_df.copy()
X = X.drop("Arrest", axis=1)
X.head(10)

,Primary Type,Location Description,Latitude,Longitude,Time_binned
0,2,145,41.815117,-87.670000,4
1,33,63,41.895080,-87.765400,3
3,18,166,41.937406,-87.716650,4
4,1,17,41.881903,-87.755121,4
5,3,145,41.744379,-87.658431,3
6,3,150,41.914636,-87.681631,5
7,33,96,41.851989,-87.689219,4
8,30,171,41.882814,-87.704326,3
10,33,135,41.763648,-87.722345,4
11,2,167,41.975968,-87.768014,3


In [48]:
# Define the target set.
y = cleaned_crime_df["Arrest"].ravel()
y[:5]

array([False, False,  True, False, False])

In [49]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.25)

In [50]:
# Creating a StandardScaler
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
#WARNING: COMPUTATIONALLY HEAVY
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=70, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [52]:
# sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3495420912880311, 'Primary Type'),
 (0.2579623386808021, 'Latitude'),
 (0.255115595464746, 'Longitude'),
 (0.10102124732388791, 'Location Description'),
 (0.03635872724253297, 'Time_binned')]

In [53]:
# Making predictions using X_test_scaled
predictions = rf_model.predict(X_test_scaled)
predictions

array([False, False, False, ..., False, False, False])

In [54]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,1111769,50747
Actual 1,127704,144431


In [55]:
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

       False       0.90      0.96      0.93   1162516
        True       0.74      0.53      0.62    272135

    accuracy                           0.88   1434651
   macro avg       0.82      0.74      0.77   1434651
weighted avg       0.87      0.88      0.87   1434651



In [56]:
accuracy_score(y_test, predictions)

0.875613650985501